In [2]:
!pip uninstall -y segmentation-models-pytorch

In [3]:
!pip install opencv-python
!pip install opencv-contrib-python

In [4]:
!pip install imutils

In [5]:
!pip install -U segmentation-models-pytorch albumentations

  Using cached segmentation_models_pytorch-0.3.2-py3-none-any.whl (106 kB)


In [6]:
!pip install ttach

In [7]:
import ttach as tta

In [1]:
import os
import cv2
import numpy as np
from PIL import Image

In [2]:
list_images = os.listdir('./all_data/images')
list_masks = os.listdir('./all_data/masks')

In [3]:
IMG_SIZE = 256

In [4]:
array_of_images = []
array_of_masks = []
for name_image, name_mask in zip(list_images, list_masks):

    image = cv2.imread(f'./all_data/images/{name_image}')
    image = np.array(Image.fromarray(image).resize((IMG_SIZE, IMG_SIZE))).astype('float32')/255
    
    img = (image - np.mean(image))/np.std(image)

    img_gray = 0.299*img[:,:,0] + 0.587*img[:,:,1] + 0.114*img[:,:,2]
    img_gray = np.moveaxis(img_gray, -1, 0)
    
    mask = cv2.imread(f'./all_data/masks/{name_mask}')*255
    mask = np.array(Image.fromarray(mask).resize((IMG_SIZE, IMG_SIZE))).astype('float32')
    mask = mask.max(axis=2)/255
#     mask = np.array([mask])
    
    #Extract our images
    array_of_images.append(img_gray)
    array_of_masks.append(mask)

In [12]:
array_of_images*=100
array_of_masks*=100

In [13]:
import matplotlib.pyplot as plt

In [14]:
import albumentations as albu

In [15]:
def get_training_augmentation():
    train_transform = [
        
        albu.RandomCrop(width = IMG_SIZE, height = IMG_SIZE),
        albu.RandomRotate90(),
        albu.Flip(p=0.5),
#         albu.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),
    ]
    return albu.Compose(train_transform)

In [16]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [17]:
class Dataset(BaseDataset):
    def __init__(
            self, 
            images, 
            masks, 
            augmentation=None
    ):
        self.images = images
        self.masks = masks
        self.augmentation = augmentation
    
    def __getitem__(self, i):
        image = self.images[i]
        mask = self.masks[i]
        # apply augmentations
        flag = np.random.rand() > 0.3
        
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            img, small_mask = sample['image'], sample['mask']
            while small_mask.sum() == 0 and flag:
                sample = self.augmentation(image=image, mask=mask)
                img, small_mask = sample['image'], sample['mask']
        
        
        return torch.tensor(np.array([img]), dtype = torch.float), torch.tensor(np.array([small_mask]), dtype = torch.float)
        
    def __len__(self):
        return len(self.images)

In [18]:
import torch
import numpy as np
import segmentation_models_pytorch as smp
from segmentation_models_pytorch import utils

In [19]:
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multicalss segmentation
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu') 

# create segmentation model with pretrained encoder
model = smp.FPN(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS,
    in_channels=1,
    classes=1,
    activation=ACTIVATION,
)

model.to(DEVICE)

FPN(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [20]:
train_dataset = Dataset(
    array_of_images, 
    array_of_masks, 
    augmentation=get_training_augmentation()
)

# valid_dataset = Dataset(
#     X_test, 
#     y_test
# )

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=8)
valid_loader = DataLoader(train_dataset, batch_size=2, shuffle=False, num_workers=8)

In [21]:
loss = smp.utils.base.SumOfLosses(
    smp.utils.losses.DiceLoss(),
    smp.utils.losses.BCELoss()
)

metrics = [
    smp.utils.metrics.IoU(threshold=0.5)
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [22]:
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model,
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [23]:
train_logs = train_epoch.run(train_loader)

train:   0%|          | 0/950 [00:00<?, ?it/s]

In [ ]:
max_score = 5
trash = 0
for i in range(0, 100):
    if trash > 6:
        break
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(valid_loader)
    
    # do something (save model, change lr, etc.)
    if max_score > valid_logs['dice_loss + bce_loss']:
        max_score = valid_logs['dice_loss + bce_loss']
        torch.save(model, './best_model.pth')
        trash = 0
        print('Model saved!')
    else:
        trash +=1
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr']*0.56
        print('Decrease decoder learning rate to 1e-5!')


Epoch: 0
train:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
# tta_model_first = tta.SegmentationTTAWrapper(torch.load('best_model.pth'), tta.aliases.d4_transform(), merge_mode='mean')